In [1]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cmocean as cmo
import os,sys,fnmatch,time
import numpy as np
import matplotlib.gridspec as gridspec
from matplotlib import animation
from salishsea_tools.nc_tools import scDataset
from salishsea_tools import (psu_tools, teos_tools)

from matplotlib import colors
from matplotlib import ticker
from matplotlib.colors import LinearSegmentedColormap
import gsw

%matplotlib inline

import sys
sys.path.append('/ocean/imachuca/Canyons/analysis_mackenzie_canyon/notebooks/general_circulation')
import general_functions

import warnings
warnings.filterwarnings("ignore")

import time
from datetime import datetime 

/home/imachuca/anaconda3/envs/gsw/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


# Functions

5 days Time elapsed (hh:mm:ss.ms) 0:03:34.994670

In [2]:
def get_vars_pressure(dirname, fname, meshmaskname, time_ind):
    
    filesU = general_functions.get_files(dirname, fname, 'grid_U') 
    filesT = general_functions.get_files(dirname, fname, 'grid_T')
    
    y,x = slice(1,-1,None), slice(1,-1,None)

    with scDataset(filesU) as dsU, scDataset(filesT) as dsT:
        vosaline0 = dsT.variables['vosaline'][:time_ind, :, y, x]
        votemper0 = dsT.variables['votemper'][:time_ind, :, y, x]
        sossheig0 = dsT.variables['sossheig'][:time_ind, y, x]
        sozotaux = dsU.variables['sozotaux'][:time_ind, 0, 0]

    with nc.Dataset(os.path.join(dirname, meshmaskname), 'r') as dsM:
        tmask0 = dsM.variables['tmask'][0, :, y, x]
        tmask_ssh0 = dsM.variables['tmask'][0, 0, y, x]
        gdept = dsM.variables['gdept_1d'][0, :]
        e3t = dsM.variables['e3t_0'][0, :, 0, 0]
        nav_lat = dsM.variables['nav_lat'][y, x]
    
    tmask = np.tile(tmask0, (len(sozotaux), 1, 1, 1)) 
    tmask_ssh = np.tile(tmask_ssh0, (len(sozotaux), 1, 1)) 

    vosaline = np.ma.array(vosaline0, mask=1 - tmask)
    votemper = np.ma.array(votemper0, mask=1 - tmask)
    sossheig = np.ma.array(sossheig0, mask=1 - tmask_ssh)
    
    return vosaline0, votemper0, sossheig0, gdept, e3t, nav_lat

In [3]:
def calculate_pressure(vosaline, votemper, sossheig, e3t):
    g = 9.81

    rho = np.full([vosaline.shape[-4], vosaline.shape[-3], vosaline.shape[-2], vosaline.shape[-1]], np.nan)
    p_hydro = np.full_like(rho, np.nan)
    p_surf = np.full([vosaline.shape[-4], vosaline.shape[-2], vosaline.shape[-1]], np.nan)
    pressure = np.full_like(rho, np.nan)

    for t in range(vosaline.shape[-4]):
        for k in range(vosaline.shape[-3]):
            gsw_vosaline = vosaline[t, k, :, :]
            gsw_votemper = votemper[t, k, :, :]
            rho[t, k, :, :] = gsw.rho(gsw_vosaline, gsw_votemper, 0)
            p_hydro[t, k, :, :] = rho[t, k, :, :] * g * np.sum(e3t[:k])

        p_surf[t, :, :] = rho[t, 0, :, :] * g * sossheig[t, :, :]
        
        pressure[t, :, :, :] = p_hydro[t, :, :, :] + p_surf[t, :, :]
        
    return rho, pressure

# Ideal

In [4]:
kind = 'ideal'
case = 'base' 
dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'
fname = '1_MCKNZ_1h_20170101_201701*'
meshmaskname = '1_mesh_mask.nc'

In [5]:
start_time = datetime.now() 

time_ind = 9*24

vosaline, votemper, sossheig, gdept, e3t, nav_lat = get_vars_pressure(dirname, fname, meshmaskname, time_ind)

rho, pressure = calculate_pressure(vosaline, votemper, sossheig, e3t)

time_elapsed = datetime.now() - start_time 
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

MemoryError: 